### Import research helpers

In [42]:
from research.research_utils import (
    get_latest_trade_analytics_key,
    expectancy
)


### 1. Must import and reload CondorResearch class
To avoid issues when class is changed
### 2. Setup condor config
### 3. Import the desired object store file


In [ ]:
# 1.
import importlib
import research.condor_research as cr
importlib.reload(cr)
CondorResearch = cr.CondorResearch

import research.config as cfg
importlib.reload(cfg)
CondorResearchConfig = cfg.CondorResearchConfig

# 2. 
cfg = CondorResearchConfig()

# 3.
import json
import pandas as pd
qb = QuantBook()

key = 'ec279b1c2a070c50a299f2a66da4aaef_2024-10-02_00-00-00_trade_analytics.json'
snp_key = 'ec279b1c2a070c50a299f2a66da4aaef_2024-10-02_00-00-00_trade_snapshots.json'

CR = None
df = None
if qb.object_store.contains_key(key):
    string_data = qb.object_store.read(f"{key}")
    json_data = json.loads(string_data)    
    df = pd.json_normalize(json_data)
    CR = CondorResearch(df, cfg)


snp_df = None
if qb.object_store.contains_key(snp_key):
    string_data = qb.object_store.read(f"{snp_key}")
    json_data = json.loads(string_data)    
    snp_df = pd.json_normalize(json_data)
    


In [ ]:
# print(df["pos.position.entry_time"].nunique() * 8)


CR.get_trades_per_day()



### Bucket Stats
* Entry Bucket
* time to adx
* time to vwap-atr

In [ ]:
CR.entry_exit_stats_group()

In [9]:
# out = CR.time_and_adx_stats()
# # Where is the strategy profitable?
# CR.pivot_view(tidy=out, index="time_bucket", columns = "adx_bucket", values="mean")
out = CR.group_stats(keys=["time_bucket", "adx_bucket"], value_col="pnl",
            include_expectancy=False)

CR.as_multiindex(out, keys=["time_bucket", "adx_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])


In [ ]:
CR.time_and_vwap_atr_stats(view="multi")

In [ ]:
# TODO : improve gap stats by using pnl per day since gap should be same for each trade during the day
# ensure you are calculating gap correctly
check = CR.df.groupby("trade_date")["gap_bucket"].nunique()
print("Days with multiple gap buckets:", (check > 1).sum())

In [ ]:
# CR.time_and_gap_pct_stats(view="multi")
CR.gap_to_daily_pnl_stats()

In [ ]:
CR.time_and_bb_position_stats(view="multi")

### VIX1D indicator shows huge tail risk so investigate why

In [50]:
CR.time_and_vix1d_stats(view="multi")

In [4]:
CR.time_and_vix1d_stats(view="multi")
# woah there is a crazy tail_ratio_q05 in these results 
mask = (
    (CR.df["time_bucket"] == pd.Interval(-0.001, 30.0, closed="right")) &
    (CR.df["vix1d_bucket"] == pd.Interval(9.81, 14.2, closed="right"))
)
sub = CR.df.loc[mask, ["pnl", "pos.position.technicals.current_vix1d", "entry_minutes_since_open"]]
display(sub["pnl"].describe(percentiles=[0.01, 0.05, 0.50, 0.95, 0.99]))
display(sub["pnl"].mean())
display(sub["pnl"].quantile(0.05))
display(sub["pnl"].nsmallest(10))
display(sub["pnl"].value_counts(bins=10).sort_index())

tidy = CR.group_stats(
    keys=["time_bucket","vix1d_bucket","vwap_atr_bucket"],
    min_count=10
)
CR.as_multiindex(tidy, keys=["time_bucket", "vix1d_bucket","vwap_atr_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])



In [ ]:
# check bins
# CR.["max_loss_norm"].quantile([0.1,0.25,0.5,0.75,0.9,0.95,0.99])


In [ ]:
# cushion norm - how much room your position has before it starts losing money, expressed relative to expected movement.
# cushion_norm ≈ (distance to short strike / expected move)
# can make cushion stricter when - move risk - is higher such as vix1d, adx, vwap_atr, gap, or just time based like my current rules in rule file
tidy = CR.group_stats(["time_bucket", "cushion_norm_bucket"], value_col="pnl")
CR.as_multiindex(tidy, keys=["time_bucket", "cushion_norm_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])


In [ ]:
# max loss norm - It’s basically your defined-risk worst case, normalized (usually by credit or some standard unit).
# again can make max loss dynamic like i mentioned for cushion
tidy = CR.group_stats(["time_bucket", "max_loss_norm_bucket"], value_col="pnl")
CR.as_multiindex(tidy, keys=["time_bucket", "max_loss_norm_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])

In [ ]:
# EM = how large price movement is relative to the option market’s expected move
tidy = CR.group_stats(["time_bucket", "em_bucket"], value_col="pnl")
CR.as_multiindex(tidy, keys=["time_bucket", "em_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])

In [ ]:
tidy = CR.group_stats(["time_bucket", "em_bucket", "cushion_norm_bucket"], value_col="pnl")
CR.as_multiindex(tidy, keys=["time_bucket", "em_bucket", "cushion_norm_bucket"], cols=["count","mean","win_rate","tail_ratio_q05","expectancy","low_confidence"])